In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from astropy.table import Table, vstack, Column
import os
import pdb
import numpy as np
import warnings

In [2]:
path = './../spectra/'
path2 = './../catalogs/'

bigFile = Table.read('bigfile.fits')

vdgcfiles = []
vugcfiles = []
for filename in os.listdir(path):
   if ((filename.endswith(".fits")) & ("spec" in filename)):
       vdgcfiles.append(filename)
   elif ((filename.endswith(".gz")) & ("spec" in filename)):
       vugcfiles.append(filename)

In [3]:
restFrame = []
velocity = []
VUGC = []
for string in vugcfiles:
    file = fits.open(path + string)
    wavelength = file[1].data['LAMBDA']
    for otherFile in bigFile:
        if (string == otherFile['SPEC1DNAME']):
            velocityVal = otherFile['VOBS']
            VUGC.append(otherFile)

            velocity.append(velocityVal)
            a = (wavelength * ((1)/(1+(velocityVal/(3e+5)))))
            restFrame.append(a)
            
restFrame

[array([[4702.636, 4703.243, 4703.849, ..., 7273.251, 7273.894, 7274.537]],
       dtype=float32),
 array([[4537.4585, 4538.332 , 4539.205 , ..., 7164.832 , 7165.537 ,
         7166.2427]], dtype=float32),
 array([[4400.4136, 4400.991 , 4401.568 , ..., 6973.8945, 6974.54  ,
         6975.1855]], dtype=float32),
 array([[4663.761 , 4664.343 , 4664.925 , ..., 7230.737 , 7231.3823,
         7232.028 ]], dtype=float32),
 array([[4458.0596, 4458.641 , 4459.2227, ..., 7032.499 , 7033.1436,
         7033.7876]], dtype=float32),
 array([[4324.8105, 4325.3745, 4325.939 , ..., 6891.3047, 6891.9497,
         6892.594 ]], dtype=float32),
 array([[4327.507 , 4328.066 , 4328.625 , ..., 6883.6665, 6884.308 ,
         6884.95  ]], dtype=float32),
 array([[4638.225 , 4638.822 , 4639.4175, ..., 7214.5747, 7215.2188,
         7215.8623]], dtype=float32),
 array([[5035.828 , 5036.4395, 5037.0513, ..., 7619.861 , 7620.5073,
         7621.1533]], dtype=float32),
 array([[5054.631 , 5055.2407, 5055.8516, ...

In [4]:
restFrame1 = []
velocity1 = []
VDGC = []
for string in vdgcfiles:
    file = fits.open(path + string)
    wavelength = file[1].data['LAMBDA']
    for otherFile in bigFile:
        s = otherFile['SPEC1DNAME']
        if (string == s[:-3]):
            VDGC.append(s[:-3])
            velocityVal = otherFile['VOBS']
            velocity1.append(velocityVal)
            a = (wavelength * ((1)/(1+(velocityVal/(3e+5)))))
            restFrame1.append(a)

VDGC

['spec1d.vdgc7.003.alg147.fits',
 'spec1d.vdgc3.011.gcA10.fits',
 'spec1d.vdgc8.065.mst532.fits',
 'spec1d.vdgc2.071.sgr788.fits',
 'spec1d.vdgc8.014.gcA21.fits',
 'spec1d.vdgc6.059.sgr417.fits',
 'spec1d.vdgc2.064.sgr747.fits',
 'spec1d.vdgc6.004.alg207.fits',
 'spec1d.vdgc2.068.sgr770.fits',
 'spec1d.vdgc6.001.alg94.fits',
 'spec1d.vdgc2.011.gcA11.fits',
 'spec1d.vdgc5.049.mst528.fits',
 'spec1d.vdgc9B.006.nuc6.fits',
 'spec1d.vdgc7.031.serendip1.fits',
 'spec1d.vdgc6.012.gcA18.fits',
 'spec1d.vdgc3.047.sgr284.fits',
 'spec1d.vdgc4B.012.gcA31.fits',
 'spec1d.vdgc2.069.sgr779.fits',
 'spec1d.vdgc3.004.alg156.fits',
 'spec1d.vdgc3.031.gcN95.fits',
 'spec1d.vdgc7.010.gcA13.fits',
 'spec1d.vdgc4B.040.sgr379.fits',
 'spec1d.vdgc2.031.gcN192.fits',
 'spec1d.vdgc1.028.gcN143.fits',
 'spec1d.vdgc3.024.gcN64.fits',
 'spec1d.vdgc9B.008.gcA9.fits',
 'spec1d.vdgc6.017.serendip1.fits',
 'spec1d.vdgc9B.002.alg86.fits',
 'spec1d.vdgc7.035.gcN160.fits',
 'spec1d.vdgc9B.039.gcN159.fits',
 'spec1d.vdg

In [5]:
warnings.simplefilter("ignore", RuntimeWarning)
def spec_interp(wv,fx,nwwv,*args):
    #Required arguments:
    #   - wv: old wavelength array
    #   - fx: flux to be rebinned, 'SPEC'
    #   - nwwv: new wavelen gth array, interval
    #
    #Optional argument: variance
    npix = len(wv)
    if len(args) == 0:
        var = np.ones(npix)
        nwvarFlag = False
    else:
        var = args[0]
        nwvarFlag = True
    nwpix = len(nwwv)
    #Calculate wavelength endpoints for each pixel
    wvl = (wv + np.roll(wv,1))/2.
    wvh = (wv + np.roll(wv,-1))/2.
    wvl[0] = wv[0] - (wv[1] - wv[0])/2.
    wvh[npix-1] = wv[npix-1] + (wv[npix-1]-wv[npix-2])/2.
    #Calculate endpoints of the final array
    bwv = np.zeros(nwpix+1)
    bwv[0:nwpix] = (nwwv+np.roll(nwwv,1))/2.
    bwv[0] = nwwv[0] - (nwwv[1] - nwwv[0])/2.
    bwv[nwpix] = nwwv[nwpix-1]+(nwwv[nwpix-1] - nwwv[nwpix - 1])/2.
    #Create tmp arrays for final array
    nwfx = np.zeros(nwpix)
    nwvar = np.zeros(nwpix)
    nwunitfx = np.zeros(nwpix)
    #Loop through the arrays
    for q in range(npix):
        #No overlap
        if (wvh[q] <= bwv[0]) | (wvl[q] >= bwv[nwpix]):
            continue
        #Find pixel that bw is within
        if wvl[q] <= bwv[0]:
            i1 = [0]
        else:
            i1 = np.argwhere((wvl[q] <= np.roll(bwv,-1)) & (wvl[q] > bwv))[0]
        if wvh[q] > bwv[nwpix]:
            i2 = [nwpix-1]
        else:
            i2 = np.argwhere((wvh[q] <= np.roll(bwv,-1)) & (wvh[q] > bwv))[0]
        j1 = i1[0]
        j2 = i2[0]
        #Now Sum up
        for kk in range(j1,j2+1):
            #Rejected pixesl do not get added in
            if var[q] > 0.:
                frac = ( np.min([wvh[q],bwv[kk+1]]) - np.max([wvl[q],bwv[kk]]) ) / (wvh[q]-wvl[q])
                nwfx[kk] = nwfx[kk]+frac*fx[q]
                nwunitfx[kk] = nwunitfx[kk]+frac*1.0
                #Variance
                if nwvarFlag:
                    if (var[q] <= 0.) | (nwvar[kk] == -1):
                       nwvar[kk] = -1
                    else:
                       nwvar[kk] = nwvar[kk]+frac*var[q]
    if nwvarFlag:
        fxOut = nwfx/nwunitfx
        varOut = nwvar*nwunitfx
        
        return fxOut,varOut
    else:
        fxOut = nwfx/nwunitfx
        return fxOut


def rebinspec(*args,**kwargs):
    #Required arguments:
    #   - wv: old wavelength array
    #   - fx: flux to be rebinned
    #   - nwwv: new wavelength array
    #
    #Optional arguments:
    #   - var = var, input and output variance
    #   - ivar = ivar, input and output ivar

    if len(args) != 3:
        print('Proper syntax is: out = rebinspec(wv, fx, nwwv, **kwargs)')
        return np.nan

    else:
        wv, fx, nwwv = args

        var = kwargs.get('var',None)
        ivar = kwargs.get('ivar',None)

        if (var is not None) & (ivar is None):
            nwfx,nwvar = spec_interp(wv,fx,nwwv,var)

            return nwfx, nwvar
        elif (var is None) & (ivar is not None):
            var = 1./ivar
            nwfx,nwvar_1 = spec_interp(wv,fx,nwwv,var)
            nwvar_1[nwvar_1 == 0.0] = -10.0
            nwivar = 1.0/nwvar_1
            nwivar[nwivar < 0.0] = 0.0
            
            return nwfx, nwivar
        else:
            nwfx = spec_interp(wv,fx,nwwv)

            return nwfx

In [12]:
for string in VDGC:
    for i in range (0, len(restFrame)):
        file = fits.open(path + string)
        correct = np.arange(4000,8750.9,0.9)
        newFlux, newVar = rebinspec(restFrame[i][0],file[1].data('SPEC'),correct, ivar = file[1].data('IVAR'))


TypeError: 'FITS_rec' object is not callable